## Extract audio features of the songs from 126 different genres.

### Save them as pickles

In [730]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing  import  StandardScaler
import numpy as np
import json
from spotipy.oauth2 import SpotifyClientCredentials
import config
import spotipy
import pickle

In [731]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=config.client_id, client_secret=config.client_secret))

In [462]:
with open('../Data/genres_top50s_tracks.json') as json_file:
    data = json.load(json_file)

In [463]:
# data = list(chain(*[data[i] for i in data.keys()]))

In [465]:
for i in [k for k in data.keys()][1:]:
    ilst=[]
    features = {i:ilst}
    for j in data[i]:
        feats = sp.audio_features(tracks=j)
        print(feats)
        ilst.append(feats)
    pickle.dump(features, open('../Data/{}_feats'.format(i), 'wb'))  

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [523]:
# os loop trough files in dir add them to the list
import os
picle_pths = []
directory = '../Data/genres feats pickles'
for filename in os.scandir(directory):
    if filename.is_file():
        picle_pths.append(filename.path)

In [524]:
def name_from_path(path):
    name = path.replace('../Data/genres feats pickles/','')
    name = name.replace('_feats','')
    return name

In [525]:
# Create the initial data frame from the first genre
path = picle_pths[0]
file = pickle.load(open(path, 'rb'))
df = pd.DataFrame([list(file.values())[0][i][0] for i in range(len(list(file.values())[0]))])
name = name_from_path(path)
df['genre']=name

In [526]:
df.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.614700,0.805600,5.200000,-5.289440,0.440000,0.099068,0.045117,0.013653,0.207456,0.523032,128.871780,230291.980000,4.000000
std,0.130793,0.123546,3.510916,1.932014,0.501427,0.087040,0.073267,0.081703,0.142737,0.232705,26.490771,40861.264702,0.202031
min,0.279000,0.469000,0.000000,-13.222000,0.000000,0.025300,0.000026,0.000000,0.033500,0.035000,76.050000,171333.000000,3.000000
25%,0.551250,0.745750,2.000000,-6.620000,0.000000,0.045150,0.003973,0.000000,0.103750,0.362250,119.981000,205474.000000,4.000000
50%,0.647000,0.815500,5.000000,-4.946000,0.000000,0.063500,0.020200,0.000007,0.165000,0.508500,126.092000,222433.500000,4.000000
75%,0.690500,0.903250,7.000000,-3.945750,1.000000,0.129500,0.049050,0.000834,0.286000,0.684000,140.038000,246566.750000,4.000000
max,0.867000,0.991000,11.000000,-1.856000,1.000000,0.466000,0.365000,0.578000,0.593000,0.967000,192.118000,413880.000000,5.000000


In [527]:
# Merge all the pickles from folder in one data frame
picle_pths.remove('../Data/genres feats pickles/.DS_Store')

In [528]:
for path in picle_pths[1:]:
    print(path)
    file = pickle.load(open(path, 'rb'))
    tmpdf = pd.DataFrame([list(file.values())[0][i][0] for i in range(len(list(file.values())[0]))])
#     create a column with genre label
    name = name_from_path(path)
    print(name)
    tmpdf['genre']=name
    df = pd.concat([df, tmpdf], ignore_index=True)

../Data/genres feats pickles/study_feats
study
../Data/genres feats pickles/malay_feats
malay
../Data/genres feats pickles/r-n-b_feats
r-n-b
../Data/genres feats pickles/power-pop_feats
power-pop
../Data/genres feats pickles/singer-songwriter_feats
singer-songwriter
../Data/genres feats pickles/iranian_feats
iranian
../Data/genres feats pickles/spanish_feats
spanish
../Data/genres feats pickles/happy_feats
happy
../Data/genres feats pickles/dub_feats
dub
../Data/genres feats pickles/latin_feats
latin
../Data/genres feats pickles/club_feats
club
../Data/genres feats pickles/soundtracks_feats
soundtracks
../Data/genres feats pickles/hardstyle_feats
hardstyle
../Data/genres feats pickles/rock_feats
rock
../Data/genres feats pickles/anime_feats
anime
../Data/genres feats pickles/funk_feats
funk
../Data/genres feats pickles/synth-pop_feats
synth-pop
../Data/genres feats pickles/party_feats
party
../Data/genres feats pickles/mandopop_feats
mandopop
../Data/genres feats pickles/techno_feats
t

In [529]:
new_cols = ['id', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo','genre']
for col in df.columns:
    if col not in new_cols:
        df.drop(col, axis=1, inplace=True)

In [530]:
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,genre
0,0.467,0.815,7,-7.929,0,0.0524,0.019800,0.000001,0.1180,0.335,140.047,75hLNJfiXV5k2YGrbGEW51,work-out
1,0.551,0.874,11,-4.559,0,0.0496,0.042700,0.001290,0.3960,0.421,125.985,4aBDS6611oBs5LBiVMuQmh,work-out
2,0.587,0.767,0,-5.713,1,0.0449,0.225000,0.000000,0.1120,0.664,100.380,5p9XWUdvbUzmPCukOmwoU3,work-out
3,0.672,0.820,11,-4.456,1,0.0459,0.036800,0.000188,0.1840,0.438,120.005,3NZJlJemX3mzjf56MqC5ML,work-out
4,0.347,0.772,5,-4.359,0,0.0549,0.059700,0.033300,0.1150,0.035,192.118,7qfwcqfGOkQYtzjF4UzJHM,work-out
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6283,0.749,0.831,10,-8.781,0,0.0537,0.002100,0.830000,0.0800,0.128,131.991,5LNkY76ZSa4NKnSJpOQhW9,trance
6284,0.741,0.630,6,-7.336,0,0.0683,0.001810,0.244000,0.2350,0.139,134.010,1HQ8F9Js4hRqsFBm3MdKmN,trance
6285,0.879,0.578,1,-9.087,1,0.0952,0.000928,0.111000,0.0447,0.181,127.989,0rzfMJGRRVmzwyj8mTZjQO,trance
6286,0.579,0.978,10,-4.309,0,0.1270,0.021900,0.068500,0.5720,0.063,129.996,3HaVjlKejQmzj6hqivJ6Kv,trance


In [531]:
df = df.reindex(columns=new_cols)

## Try to predict the genre

In [623]:
scaler = StandardScaler()
scaler.fit(df[new_cols[1:-1]])

StandardScaler()

In [682]:
df[new_cols[1:-1]] = scaler.transform(df[new_cols[1:-1]])

In [683]:
df

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,75hLNJfiXV5k2YGrbGEW51,-0.521102,0.599859,0.487627,0.028902,-1.349912,-0.287920,-0.774052,-0.539563,-0.486856,-0.576797,0.597017,work-out
1,4aBDS6611oBs5LBiVMuQmh,-0.035885,0.838053,1.612014,0.742731,-1.349912,-0.316576,-0.703121,-0.535438,1.110905,-0.250915,0.101268,work-out
2,5p9XWUdvbUzmPCukOmwoU3,0.172065,0.406074,-1.480049,0.498292,0.740789,-0.364677,-0.138454,-0.539567,-0.521340,0.669889,-0.801424,work-out
3,3NZJlJemX3mzjf56MqC5ML,0.663058,0.620044,1.612014,0.764548,0.740789,-0.354443,-0.721396,-0.538965,-0.107531,-0.186497,-0.109554,work-out
4,7qfwcqfGOkQYtzjF4UzJHM,-1.214268,0.426260,-0.074566,0.785094,-1.349912,-0.262334,-0.650464,-0.432992,-0.504098,-1.713593,2.432755,work-out
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6283,5LNkY76ZSa4NKnSJpOQhW9,1.107839,0.664454,1.330917,-0.151568,-1.349912,-0.274615,-0.828877,2.116819,-0.705255,-1.361186,0.313007,trance
6284,1HQ8F9Js4hRqsFBm3MdKmN,1.061628,-0.147021,0.206531,0.154510,-1.349912,-0.125195,-0.829775,0.241347,0.185583,-1.319504,0.384186,trance
6285,0rzfMJGRRVmzwyj8mTZjQO,1.858770,-0.356954,-1.198952,-0.216384,0.740789,0.150108,-0.832507,-0.184315,-0.908136,-1.160352,0.171918,trance
6286,3HaVjlKejQmzj6hqivJ6Kv,0.125854,1.257920,1.330917,0.795685,-1.349912,0.475558,-0.767548,-0.320335,2.122436,-1.607492,0.242674,trance


In [723]:
avg_feats_byGenre = df.groupby(['genre']).mean()
avg_feats_byGenre.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
count,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000,126.000000
mean,-0.000150,0.001260,-0.000241,0.000092,0.000300,0.015006,0.002993,-0.001004,0.005687,-0.000857,-0.001401
std,0.630771,0.689181,0.155402,0.669473,0.292164,0.836241,0.698494,0.644806,0.408944,0.578753,0.309463
min,-1.909918,-2.245079,-0.288199,-3.564982,-0.931772,-0.463786,-0.817862,-0.539562,-0.459119,-1.335858,-0.799445
25%,-0.350380,-0.433701,-0.112514,-0.267776,-0.220933,-0.260533,-0.569351,-0.468695,-0.193839,-0.440044,-0.195450
50%,0.056479,0.132191,-0.001481,0.193112,0.009044,-0.125758,-0.199878,-0.316302,-0.048184,-0.002893,0.015847
75%,0.429547,0.502159,0.098309,0.426692,0.197207,0.087750,0.483992,0.328092,0.080096,0.382572,0.197220
max,1.289218,1.209474,0.470762,1.024636,0.615347,8.607088,2.013348,2.021874,3.258494,1.289746,1.403746


### Get new song feats and preprocess

In [724]:
idd = '7nnWIPM5hwE3DaUBkvOIpy'
song = sp.audio_features(tracks=idd)[0]
song

{'danceability': 0.645,
 'energy': 0.792,
 'key': 4,
 'loudness': -7.34,
 'mode': 0,
 'speechiness': 0.036,
 'acousticness': 0.0562,
 'instrumentalness': 0.0121,
 'liveness': 0.119,
 'valence': 0.666,
 'tempo': 90.565,
 'type': 'audio_features',
 'id': '7nnWIPM5hwE3DaUBkvOIpy',
 'uri': 'spotify:track:7nnWIPM5hwE3DaUBkvOIpy',
 'track_href': 'https://api.spotify.com/v1/tracks/7nnWIPM5hwE3DaUBkvOIpy',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7nnWIPM5hwE3DaUBkvOIpy',
 'duration_ms': 232453,
 'time_signature': 4}

In [725]:
v_song = [val for key,val in song.items() if key in new_cols[1 :-1]]
len(v_song)
a_song = np.array(v_song)
r_song = a_song.reshape(1, -1)
s_song = scaler.transform(r_song)
new = avg_feats_byGenre.loc[:,['danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']] - s_song

/Users/danielfriedman/miniconda3/envs/ih_da/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [726]:
new['sum']=abs(new.sum(axis = 1))
new

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,sum
genre,,,,,,,,,,,,
acoustic,-0.542287,-1.496326,0.314828,-0.664564,1.756189,0.051253,1.462507,0.077621,0.630920,-0.991817,0.951783,1.550107
afrobeat,0.192816,-0.428022,0.517218,-0.124240,1.128979,0.329053,1.132092,0.462178,0.341093,0.274423,1.023142,4.848730
alt-rock,-0.847858,-0.067017,0.399157,0.131946,1.588933,0.199466,0.090713,0.418837,0.275803,-0.575825,1.279393,2.893548
alternative,-0.721586,-0.005087,0.185524,0.329696,1.547119,0.210806,0.007502,0.342108,0.253653,-0.743844,1.231586,2.637477
ambient,-1.299837,-1.562189,0.089951,-1.787427,1.338049,0.115995,1.344977,2.408478,0.467592,-1.778275,0.718835,0.056149
...,...,...,...,...,...,...,...,...,...,...,...,...
trance,-0.180339,0.017844,0.410401,0.058644,0.919908,0.224008,-0.122663,1.589843,0.650725,-1.296736,1.478604,3.750241
trip-hop,-0.257973,-0.736948,0.157414,-0.479091,1.045350,0.324529,0.588699,1.514991,0.291631,-0.963541,1.029318,2.514380
turkish,-0.149839,-0.334844,0.247365,0.194310,0.669024,0.465250,0.608323,0.026464,0.239928,-0.363131,1.112518,2.715369


In [729]:
new.loc[new['sum'].idxmin(),:]

danceability       -0.933337
energy             -1.527927
key                 0.174280
loudness           -0.912773
mode                1.672561
speechiness         0.168743
acousticness        1.368748
instrumentalness    0.443343
liveness            0.069439
valence            -1.444027
tempo               0.947178
sum                 0.026229
Name: sleep, dtype: float64

In [479]:
df.insert(0, 'setnumber', setnumber)
df.insert(1, 'Title', ['dummy' for i in range(6288)])
df.to_csv('../Data/genres_a-feats.csv')

#### Recheck the file

In [482]:
df2 = pd.read_csv('../Data/genres_a-feats.csv')

In [483]:
df2.drop('Unnamed: 0', axis=1)

,setnumber,Title,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,0,dummy,75hLNJfiXV5k2YGrbGEW51,0.467,0.815,7,-7.929,0,0.0524,0.019800,0.000001,0.1180,0.335,140.047,trance
1,1,dummy,4aBDS6611oBs5LBiVMuQmh,0.551,0.874,11,-4.559,0,0.0496,0.042700,0.001290,0.3960,0.421,125.985,trance
2,2,dummy,5p9XWUdvbUzmPCukOmwoU3,0.587,0.767,0,-5.713,1,0.0449,0.225000,0.000000,0.1120,0.664,100.380,trance
3,3,dummy,3NZJlJemX3mzjf56MqC5ML,0.672,0.820,11,-4.456,1,0.0459,0.036800,0.000188,0.1840,0.438,120.005,trance
4,4,dummy,7qfwcqfGOkQYtzjF4UzJHM,0.347,0.772,5,-4.359,0,0.0549,0.059700,0.033300,0.1150,0.035,192.118,trance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6283,6283,dummy,5LNkY76ZSa4NKnSJpOQhW9,0.749,0.831,10,-8.781,0,0.0537,0.002100,0.830000,0.0800,0.128,131.991,NaN
6284,6284,dummy,1HQ8F9Js4hRqsFBm3MdKmN,0.741,0.630,6,-7.336,0,0.0683,0.001810,0.244000,0.2350,0.139,134.010,NaN
6285,6285,dummy,0rzfMJGRRVmzwyj8mTZjQO,0.879,0.578,1,-9.087,1,0.0952,0.000928,0.111000,0.0447,0.181,127.989,NaN
6286,6286,dummy,3HaVjlKejQmzj6hqivJ6Kv,0.579,0.978,10,-4.309,0,0.1270,0.021900,0.068500,0.5720,0.063,129.996,NaN


#### Can play a song on a device. Requires premium.

In [741]:
var='5fY5gnb48X4m80sPTz6GLR'
result = sp.recommendations(seed_tracks=[var], limit=10, market='US')
result['tracks'][0]['id']

In [748]:
result['tracks'][0].keys()

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [749]:
result['tracks'][0]['id']

'0bcGY7mfG10QfeaDbz97hC'

In [411]:
sp.start_playback(uris=['spotify:track:7lEptt4wbM0yJTvSG5EBof'], device_id='18:3e:ef:cd:d3:74')

HTTP Error for PUT to https://api.spotify.com/v1/me/player/play?device_id=18:3e:ef:cd:d3:74 with Params: {} returned 403 due to Player command failed: Premium required


SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/me/player/play?device_id=18:3e:ef:cd:d3:74:
 Player command failed: Premium required, reason: PREMIUM_REQUIRED